In [ ]:
import multiprocessing

def compute_res_rms(k):
    T_env = 200 
    
    idx_train_1 = (t < 300-10*k)
    idx_train_2 = (t > 300+10*k) 
    idx_train = idx_train_1|idx_train_2
    t_train = t[idx_train]
    y_train = y[idx_train]
    
    gp = GaussianProcess(kernel, t_train)
    cond_gp = gp.condition(y_train, t).gp
    pred_mean, pred_std = cond_gp.loc, cond_gp.variance**0.5   
    
    fig, axs = plt.subplots(3, 1, figsize=(16, 6))
    plt.subplots_adjust(hspace=0.3)

    for i in range(3):
        idx_t = ((t>=i*T_env) & (t<(i+1)*T_env))
        axs[i].plot(t[idx_t], y[idx_t], 'ko', alpha=0.3)
        axs[i].plot(t[idx_t&idx_train_1], pred_mean[idx_t&idx_train_1], color='k', alpha=0.8)
        axs[i].plot(t[idx_t&idx_train_2], pred_mean[idx_t&idx_train_2], color='k', alpha=0.8)
        axs[i].plot(t[idx_t&~idx_train], pred_mean[idx_t&~idx_train], color='r', alpha=0.8)
        axs[i].fill_between(t[idx_t], pred_mean[idx_t]+pred_std[idx_t], pred_mean[idx_t]-pred_std[idx_t], 
                            color='r', alpha=0.3, edgecolor="none")
        axs[i].set_ylabel('RV [m/s]')
        axs[i].set_xlim([i*T_env, (i+1)*T_env])
        axs[i].set_ylim([-3.5, 3.5])    
        axs[i].grid(True)
    axs[-1].set_xlabel('$t$ [minutes]')
    plt.savefig('./Figure/GP_recovery_run-19_train-test/' + str(k) + '.png')
    plt.show()
    return np.std(pred_mean[~idx_train] - y[~idx_train])

# Create a list of items to process
items = [1, 2]

# Create a Pool object with 4 worker processes
pool = multiprocessing.Pool(processes=2)

# Apply the compute function to each item in parallel
results = pool.map(compute_res_rms, items)

# Print the results
print(results)
